<img src="media/docker_logo_1.png" width="500" height="500">

It is probably fair to say that access to servers has never been easier.  With platforms such as AWS, Azure, and Google GCE we can now launch on-demand servers of all varieties and configurations.  This programable infrastructure (IaaS) help companies, agencies, and institutions maintain agility as market and mission pressures evolve. However, even with the rise of IaaS, application packaging, configuration, and composition has not advanced despite considerable efforts in configuration management.  This is where [`docker`](https://www.docker.com/) comes in.  

`Docker` is not about full virtualization but rather about the ease of packaging and running applications using [Linux containers](https://en.wikipedia.org/wiki/LXC).  The idea is that `docker` containers wrap a piece of software or application in a complete filesystem that contains everything needed to run: code, runtime, system tools, system libraries (i.e. anything that can be installed on a server). This guarantees that the software will always run the same everywhere, regardless of the OS/compute environment running the container. Docker also provides portable Linux deployment such that containers can be run on any Linux system with kernel is 3.10 or later.  All major Linux distros have supported Docker since 2014.  While no doubt containers and virtual machines have similar resource isolation and allocation benefits, the architectural approach of Linux containers allows containerized applications to be more portable and efficient.

At NVIDIA, we use containers in a variety of ways including development, testing, benchmarking, and of course in production as the mechanism for deploying deep learning frameworks. Using [`nvidia-docker`](https://github.com/NVIDIA/nvidia-docker), a light-weight `docker` plugin, we can develop and prototype GPU applications on a workstation, and then deploy those applications anywhere that supports GPU containers.

# Setup
In the interest of time, we've already configured docker and nvidia-docker. If you're interested in setup details, see Appendix A at the bottom.

# First Contact

The simplest way to interact with `docker` is probably to just ask for the version information

In [11]:
docker --version

Docker version 1.12.3, build 6b644ec


We can ask `nvidia-docker` for the version information too

In [12]:
nvidia-docker --version

Docker version 1.12.3, build 6b644ec


Notice that `nvidia-docker` invocation here was simply "pass through" to `docker` command itself.

Next best way to get familiar with docker command line is to ask for `--help`

In [13]:
docker --help

Usage: docker [OPTIONS] COMMAND [arg...]
       docker [ --help | -v | --version ]

A self-sufficient runtime for containers.

Options:

  --config=~/.docker              Location of client config files
  -D, --debug                     Enable debug mode
  -H, --host=[]                   Daemon socket(s) to connect to
  -h, --help                      Print usage
  -l, --log-level=info            Set the logging level
  --tls                           Use TLS; implied by --tlsverify
  --tlscacert=~/.docker/ca.pem    Trust certs signed only by this CA
  --tlscert=~/.docker/cert.pem    Path to TLS certificate file
  --tlskey=~/.docker/key.pem      Path to TLS key file
  --tlsverify                     Use TLS and verify the remote
  -v, --version                   Print version information and quit

Commands:
    attach    Attach to a running container
    build     Build an image from a Dockerfile
    commit    Create a new image from a container's changes
    cp        Copy files/folde

The format of `docker` command line interactions is: 

`docker [OPTIONS] COMMAND [arg...]` 

and as the help display shows there are a lot of commands to choose from.  Don't worry, much like a big city, once we become more familiar with these commands the list won't seem so big.  We can start to drill down and get help that is specific to each command.  For example, one of the most useful `docker` commands is **`images`** which list all local containers on the host that `docker` knows about 

In [15]:
docker images --help


Usage:	docker images [OPTIONS] [REPOSITORY[:TAG]]

List images

Options:
  -a, --all             Show all images (default hides intermediate images)
      --digests         Show digests
  -f, --filter value    Filter output based on conditions provided (default [])
      --format string   Pretty-print images using a Go template
      --help            Print usage
      --no-trunc        Don't truncate output
  -q, --quiet           Only show numeric IDs


OK, lets now ask `docker` about the what container images are available locally on the host

In [10]:
docker images

REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE
nvidia/cuda         8.0-cudnn5-devel    31582c303549        5 weeks ago         1.776 GB
nvidia/cuda         latest              367795fb1051        5 weeks ago         1.615 GB
hello-world         latest              c54a2cc56cbb        5 months ago        1.848 kB


Here the output specifies three container images with some general metadata associated with each one.  First you'll notice that the images are quite large on average (~ 2GB) and that each image is associated with a unique ID hash.  When containers are created (i.e. via the **`create`** command) they are created from images.  There is no limit to the number of containers we can create from an images so it is important that `docker` associates UUIDs for each image and container.  Notice the `REPOSITORY` and `TAG` columns here specify more human readable image labels.  The repository loosely coresponds to the image name (i.e. url) and just as in the version control system [GIT](https://git-scm.com/) images can be modified and "tagged" rather than explicitly changing the image name for each image version. 

Here we have the "`nvidia/cuda`" container with ID `c54a2cc56cbb` and is tagged as the "`latest`" version of the image (i.e. most current).  The deep learning library [`cuDNN`](https://developer.nvidia.com/cudnn) was added to the image and a new image was created under the same name but tagged appropriately as "`8.0-cudnn5-devel`".

You're probably wondering already "*where does docker store these containers?*".  In general, docker works in `/var/lib/docker` and images are stored in `image` subdirectory.  For more information and details about where and how docker stores images on the host machine, see [here](https://stackoverflow.com/questions/19234831/where-are-docker-images-stored-on-the-host-machine#). 

For now just know that docker works with "images" and all containers are created from these images.  We will go into all the details about creating and modifying images etc in just a bit.  But first, lets actually kick around some containers!

## Getting Started with Containers

First things first lets have docker list *all* containers using the **`ps`** command

In [19]:
docker ps -a

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES


Again, don't forget you can get help for each command with `docker [COMMAND] --help`.  Use this to get additional details on the **`ps`** command.

Lets now use the docker command **`create`** to initialize a container from the `nvidia/cuda:latest` image

In [20]:
docker create nvidia/cuda:latest

9a12d94b12fd4e3de47a3ee3bab4d771d80b6895b575dc2c1ad257cb0f3d121f


The responce we recieved is a sha256 UUID for the generated container and listing `docker` containers again we see this new container now listed 

In [21]:
docker ps -a

CONTAINER ID        IMAGE                COMMAND             CREATED             STATUS              PORTS               NAMES
9a12d94b12fd        nvidia/cuda:latest   "/bin/bash"         45 seconds ago      Created                                 lonely_easley


It is important to understand that the container is not actually doing anything right now.  We've only "stamped" out a container from an image -- the container is not running at this point.  Were the container active the `STATUS` would read "running".  OK, so what is the container doing there?  Well the answer is "nothing".  Think about when we enter commands on the command-line -- each time we hit enter we implicitly specify that we would like that command to be executed immediately.  You can think of containers as a command that has not yet executed.  This command is wrapped up in the container and has all the resources (libraries etc) needed for successful execution.  Speaking of which, lets actually run this container ... 

In [22]:
nvidia-docker run 9a12d94b12fd

Using default tag: latest
Pulling repository docker.io/library/9a12d94b12fd
nvidia-docker | 2016/11/30 01:01:55 Error: image library/9a12d94b12fd:latest not found


hm ... using the **`run`** command seemed like a good guess.  Lets try the **`start`** command instead

In [23]:
nvidia-docker start 9a12d94b12fd

9a12d94b12fd


OK, that looks better. Using the **`start`** command docker returned the sha256 UUID.  Lets have a look at the docker containers again

In [27]:
docker ps -a

CONTAINER ID        IMAGE                COMMAND             CREATED             STATUS                     PORTS               NAMES
9a12d94b12fd        nvidia/cuda:latest   "/bin/bash"         About an hour ago   Exited (0) 6 minutes ago                       lonely_easley


Now the status says "Exited (0) ...".  Notice that the command (i.e. entry point) is `/bin/bash`.  When the **`start`** was issued the "COMMAND" was executed and by definition *bash command language interpreter that executes commands read from the standard input or from a file*.  However, there were no commands to execute from standard input!  Containers can have other entry points -- the reason `/bin/bash` is used most often is that it allows the container to act more generically as a shell so we can send it additional instructions. Note that all containers have a default entrypoint of "`/bin/sh -c`" unless otherwise specified.

Here our hands are tied with what the container will do.  Each time we issue the **`start`** command the container executes the entrypoint "`/bin/bash`" and since there is nothing on standard input the container simply exits.  This is where the **`run`** command comes in.

Instead of creating and starting a container explicitly we can use the **`run`** command to exectue a command within a particular image via creating a container from that image with the appropriate entrypoint.  Lets issue a **`run`** command passing the image ID of the "`nvidia/cuda:latest`" image as the argument.

In [31]:
nvidia-docker run 367795fb1051

Notice that the command **`start`** takes a container ID as the argument while the **`run`** command takes an image ID.  Lets have a look at the containers

In [33]:
docker ps -a

CONTAINER ID        IMAGE                COMMAND             CREATED              STATUS                          PORTS               NAMES
b79dee54ffe1        367795fb1051         "/bin/bash"         About a minute ago   Exited (0) About a minute ago                       amazing_euler
ff763f6099b1        367795fb1051         "/bin/bash"         3 minutes ago        Exited (0) 2 minutes ago                            suspicious_pike
9a12d94b12fd        nvidia/cuda:latest   "/bin/bash"         2 hours ago          Exited (0) 18 minutes ago                           lonely_easley


Now we have an additional container both from image `367795fb1051` which have exited.  At this point the **`run`** command has done exactly what the **`start`** command has done (i.e. started a container which executed the entrypoint and exited).  However, the docker **`run`** command allows us to pass an alternative command to the container (`docker run --help`).  Lets try to pass an alternative instruction.

In [34]:
nvidia-docker run 367795fb1051 nvidia-smi

Wed Nov 30 01:49:50 2016       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 367.57                 Driver Version: 367.57                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GRID K520           Off  | 0000:00:03.0     Off |                  N/A |
| N/A   26C    P8    17W / 125W |      0MiB /  4036MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

Finally!  Just to be clear, the `nvidia-smi` command was exectued within the container -- not the host.  Lets have a look at the containers yet again

In [36]:
docker ps -a

CONTAINER ID        IMAGE                COMMAND             CREATED             STATUS                      PORTS               NAMES
2747af7b70a0        367795fb1051         "nvidia-smi"        19 seconds ago      Exited (0) 18 seconds ago                       berserk_goldwasser
b79dee54ffe1        367795fb1051         "/bin/bash"         7 minutes ago       Exited (0) 7 minutes ago                        amazing_euler
ff763f6099b1        367795fb1051         "/bin/bash"         9 minutes ago       Exited (0) 9 minutes ago                        suspicious_pike
9a12d94b12fd        nvidia/cuda:latest   "/bin/bash"         2 hours ago         Exited (0) 24 minutes ago                       lonely_easley


We now have a new container from image `367795fb1051` but the "COMMAND" has been set to `nvidia-smi` as instructed by our **`run`** command.  Just for kicks lets issue a **`start`** command to this new container.  Each container gets a uuid that will change every time this lab is run so make sure to replace the container ID in the command below with the appropriate container ID listed above.

In [39]:
docker start 2747af7b70a0

2747af7b70a0


Now wait a minute, where is our output??

In [40]:
docker ps -a

CONTAINER ID        IMAGE                COMMAND             CREATED             STATUS                      PORTS               NAMES
2747af7b70a0        367795fb1051         "nvidia-smi"        11 minutes ago      Exited (0) 3 seconds ago                        berserk_goldwasser
b79dee54ffe1        367795fb1051         "/bin/bash"         19 minutes ago      Exited (0) 19 minutes ago                       amazing_euler
ff763f6099b1        367795fb1051         "/bin/bash"         20 minutes ago      Exited (0) 20 minutes ago                       suspicious_pike
9a12d94b12fd        nvidia/cuda:latest   "/bin/bash"         2 hours ago         Exited (0) 36 minutes ago                       lonely_easley


Sure enough when we check the docker container status it has status of "Exited (0) 10 seconds ago ..." which means that the **`start`** command did indeed start the container. The long story short is that the **`run`** command automatically provides the standard output from the command specified where as **`start`** does not forward the stdout by default -- we have to explicitly ask.  According to the help section for the **`start`** command, the option "`--attach`" attaches STDOUT/STDERR and forward signals. 

In [45]:
docker start --attach 2747af7b70a0

Wed Nov 30 02:10:54 2016       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 367.57                 Driver Version: 367.57                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GRID K520           Off  | 0000:00:03.0     Off |                  N/A |
| N/A   26C    P8    17W / 125W |      0MiB /  4036MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

Bingo!

A few final words on starting and running containers. Keep an eye out on the container list when using the **`run`** command as each invocation creates a *new* image.  There is no problem having many (many) container stitting around but eventually it creates clutter.  Remember, containers are ment to be light-weight and disposable.  To that end lets clean up our containers.

In [51]:
# generate a list of container ID from the docker ps command
docker ps -a | awk '{print $1}' | tail -n +2

2747af7b70a0
b79dee54ffe1
ff763f6099b1
9a12d94b12fd


In [62]:
# for each container ID use the docker "rm" command to remove/delete the container
for cid in $(docker ps -a | awk '{print $1}' | tail -n +2);do docker rm $cid; done

0d680aedcfd5
8a4cff61a952
bf3cf6116d81
00fa8fd7279d
b1a17b9a0c02


All cleaned up.

So, the **`run`** command creates a new container each time and using the docker **`ps`** command we can see each new container as we run commands.  However this can get combersom to have to manually clean out containers all the time. The solution to this is to use the **`--rm`** option with the **`run`** command.  This instructs docker to simply remove the container after execution.  This is quite convenient for most situations.  Keep in mind however, that once the container has been deleted it can not be started again etc -- it's gone.  In general this is the desired workflow since containers are intended to be light-weight disposible execution units.  After all, if you need the container again, no problem, just create another one! 

### Summary
So far we have discussed what docker is (i.e. virtual machines v.s. Linux containers) and how to view (**`ps`**), **`create`**, **`start`**, **`run`**, and **`rm`** containers created from `docker` images.  Furthermore we've investigated various options associated with these `docker` commands such as `--attach` and `--rm` and familiarized ourselves with how to obtain help for `docker` and each of the `docker` commands.

### Exercises
Make sure that you're comfortable creating containers and executing commands before moving forward.  Docker is quite forgiving so do be afraid to try lots of different things out while you explore.  Here are a few suggestions: 
0. Try launching containers from the other images
1. Try issuing the **`run`** command with the option `--rm` and confirm the continer is cleaned up 
2. It is often useful to give our containers a name, try this with the `--name` option with the **`run`** command
3. Have the container execute `whoami`.  Think about what user might get returned before you run this.
4. Maybe try `ifconfig` inside of the container.  Is the MAC address the same every time?
5. Get the container to ping google (pro tip: use option `-c1` so you don't ping forever) 
6. What does the container return when asked for disk usage (i.e `df -h`)??
7. Use the `--env` option with the **`run`** command to set environment variables AWS_S3_BUCKET, AWS_ACCESS_KEY, and AWS_SECRET_KEY
8. Notice the containers maintain state. Verify this by touching a file and then running the container again


#### Food for Thought: 
What happens when you execute `rm -rf /` inside a container?


# Diving Deeper into Images
By now you probably comfortable with launching containers with `docker` from the images that were already available when we started.  The next step is understanding how to manage your own images.  This includes things like importing images into `docker`, modifying existing images, exporting images, and of course deleting images. 

In the `docker` world most images have a "parent".  This means that the image was created by modifying some existing image.  In general, this is the typical workflow in `docker`.  The idea is that it is easy to create images and lets just reuse what's already existing so as to be most efficient.  

However, there is an essential difference when working with `docker` images. In the virtual machine world, you modify a 4 GB machine image and then do "save as" and create a new 4 GB machine image that contains your changes/updates.  In this way virtual machine images are totally independent but very heavy weight. 

When we make modification to an existing `docker` image and use this to create our own "new" image, `docker` does not store two images.  Just a the GIT version control does not make a new copy of a file everytime a modification is commited, so too `docker` works with images in "layers" so that changes or modifications to an image are stored as a new light-weight image deltas called "layers".  In this way we can take an existing 2 GB `docker` image and create 10 new images each with a few modifications of this base image without having to store 20 GB of new images!  That is, in creating a new docker image from a parent, we only have to store the changes to the parent image.

As you might imagine, only having to store deltas to images allows for many many (many!) images to be generated without having to pay the full cost of having all those images around.  Therefore in the `docker` world, images abound since it is efficient and light-weight to generate new images from an existing parent image. Luckily, `docker` provides ways to manage all these images using "image repositories" so that we can manage images just like we would version controled files in a `git` repository.  More on `docker` repositories later.

### New Image from Container Modifications
Lets first update the existing `nvidia/cuda` image by creating a container that executes `apt-get update`.

In [63]:
docker images

REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE
nvidia/cuda         8.0-cudnn5-devel    31582c303549        5 weeks ago         1.776 GB
nvidia/cuda         latest              367795fb1051        5 weeks ago         1.615 GB
hello-world         latest              c54a2cc56cbb        5 months ago        1.848 kB


In [96]:
nvidia-docker run 367795fb1051 apt-get update

Ign http://archive.ubuntu.com trusty InRelease
Get:1 http://archive.ubuntu.com trusty-updates InRelease [65.9 kB]
Get:2 http://archive.ubuntu.com trusty-security InRelease [65.9 kB]
Ign http://developer.download.nvidia.com  InRelease
Get:3 http://developer.download.nvidia.com  Release.gpg [819 B]
Get:4 http://developer.download.nvidia.com  Release [564 B]
Get:5 http://archive.ubuntu.com trusty Release.gpg [933 B]
Get:6 http://archive.ubuntu.com trusty Release [58.5 kB]
Get:7 http://developer.download.nvidia.com  Packages [107 kB]
Get:8 http://archive.ubuntu.com trusty-updates/main Sources [478 kB]
Get:9 http://archive.ubuntu.com trusty-updates/restricted Sources [5921 B]
Get:10 http://archive.ubuntu.com trusty-updates/universe Sources [214 kB]
Get:11 http://archive.ubuntu.com trusty-updates/main amd64 Packages [1155 kB]
Get:12 http://archive.ubuntu.com trusty-updates/restricted amd64 Packages [20.4 kB]
Get:13 http://archive.ubuntu.com trusty-updates/universe amd64 Packages [503 kB]
Get

Ok, since we did not use the `--rm` option our container is still available -- which is what we want since we're going to create a new image from this updated container. Notice that we can not save changes to a container that has been removed/deleted.  This should be obvious but just saying ... :)

In [97]:
docker ps -a

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS                         PORTS               NAMES
65b19c64b404        367795fb1051        "apt-get update"    12 seconds ago      Exited (0) 4 seconds ago                           trusting_goodall
7076186eb1f0        367795fb1051        "apt-get update"    About an hour ago   Exited (0) About an hour ago                       focused_ardinghelli


 `Docker` lets us keep these changes by committing them into a new image.  Under the hood, `docker` keeps track of the differenced between the base image (`nvidia/cuda` or rather `367795fb1051`) by creating a new image layer using the union filesystem ([UnionFS](https://en.wikipedia.org/wiki/UnionFS)).  To see this, we can inspect the changes to the container using the `docker` **`diff`** command which takes a the container ID as an argument.  

In [98]:
docker diff 65b19c64b404

C /usr
C /usr/local
A /usr/local/nvidia
C /var
C /var/lib
C /var/lib/apt
C /var/lib/apt/lists
A /var/lib/apt/lists/archive.ubuntu.com_ubuntu_dists_trusty-security_main_source_Sources.gz
A /var/lib/apt/lists/archive.ubuntu.com_ubuntu_dists_trusty-updates_main_source_Sources.gz
A /var/lib/apt/lists/developer.download.nvidia.com_compute_cuda_repos_ubuntu1404_x86%5f64_Release.gpg
A /var/lib/apt/lists/partial
A /var/lib/apt/lists/archive.ubuntu.com_ubuntu_dists_trusty-security_restricted_source_Sources.gz
A /var/lib/apt/lists/archive.ubuntu.com_ubuntu_dists_trusty-updates_main_binary-amd64_Packages.gz
A /var/lib/apt/lists/developer.download.nvidia.com_compute_cuda_repos_ubuntu1404_x86%5f64_Release
A /var/lib/apt/lists/archive.ubuntu.com_ubuntu_dists_trusty-updates_universe_binary-amd64_Packages.gz
A /var/lib/apt/lists/archive.ubuntu.com_ubuntu_dists_trusty_Release.gpg
A /var/lib/apt/lists/archive.ubuntu.com_ubuntu_dists_trusty_restricted_source_Sources.gz
A /var/lib/apt/lists/archive.ubuntu

where `A` means that the file or directory listed was added, `C` means created and `D` means deleted

Lets now use the `docker` **`commit`** command to generate a new image from this container.  You might want to check your disk usage before and after image creation just to verify for yourself that the new image does not eat up an additional 2 GB of disk space on the host.

In [68]:
df -h

Filesystem      Size  Used Avail Use% Mounted on
udev            7.4G   12K  7.4G   1% /dev
tmpfs           1.5G  800K  1.5G   1% /run
/dev/xvda1       32G  8.4G   22G  28% /
none            4.0K     0  4.0K   0% /sys/fs/cgroup
none            5.0M     0  5.0M   0% /run/lock
none            7.4G     0  7.4G   0% /run/shm
none            100M   16K  100M   1% /run/user


In [124]:
docker commit 65b19c64b404 newiamgename:update

sha256:5980494cc2123a15bb4ed8f5393d847332a791f23fa237bfc37975bc328d3921


Here using the `docker` **`commit`** command we provided the unique container ID as provided by the **`ps`** command and a new name:tag for the resulting image.  Now lets list the `docker` images and we should see our new image there.

In [125]:
docker images

REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE
newiamgename        update              5980494cc212        5 seconds ago       1.637 GB
nvidia/cuda         8.0-cudnn5-devel    31582c303549        5 weeks ago         1.776 GB
nvidia/cuda         latest              367795fb1051        5 weeks ago         1.615 GB
hello-world         latest              c54a2cc56cbb        5 months ago        1.848 kB


Again, notice that the image size says something like 1.6 GB.  Verify with `df` that we have not actually used additional physical space on host disk in generating this image

In [73]:
df -h

Filesystem      Size  Used Avail Use% Mounted on
udev            7.4G   12K  7.4G   1% /dev
tmpfs           1.5G  800K  1.5G   1% /run
/dev/xvda1       32G  8.4G   22G  28% /
none            4.0K     0  4.0K   0% /sys/fs/cgroup
none            5.0M     0  5.0M   0% /run/lock
none            7.4G     0  7.4G   0% /run/shm
none            100M   16K  100M   1% /run/user


**GOTCHA**: `docker` does not allow upper case characters in the image names and doing so generates the error message: "invalid reference format" 

### Gernerating Tar Files for Sharing
There are two `docker` commands for creating a `tar` file that can be shared with others.  The first is that we can use the `docker` commands **`save`** and **`load`** to create and ingest *image* tar files.  The second option is to use the `docker` commands **`export`** and **`import`** to create and ingest *container* tar files

Notice the `help` definitions for each set of commands:

|  | When working with Containers   |
|------|------|
| **`export`** | Export a container's filesystem as a tar archive                         |
| **`import`** | Import the contents from a tarball to create a filesystem image |
|              | **When working with Images** |
| **`save`**   | Save one or more images to a tar archive (streamed to STDOUT by default)   |
| **`load`**   | Load an image from a tar archive or STDIN|

Lets save the new image created in the previous section as a tar-ball on the file system.

In [126]:
docker save -o dockerimageexport.tar newiamgename:update

Now if we look in our current working directory we should see a nice fat tar-ball of our `docker` image

In [102]:
ls -lah dockerimageexport.tar

-rw------- 1 ubuntu ubuntu 1.6G Nov 30 18:31 dockerimageexport.tar


Keep in mind that when an image is saved to the host file system the full size of the image is physically allocated. We can see this here as the file `dockerimageexport.tar` has size 1.6G.

Lets now remove our new image we just commited from `docker` using the **`rmi`** command

In [127]:
docker rmi 5980494cc212

Untagged: newiamgename:update
Deleted: sha256:5980494cc2123a15bb4ed8f5393d847332a791f23fa237bfc37975bc328d3921
Deleted: sha256:9a5db72eda6a4bebd250ce33ee0e6c24866e8f519de2c34c2e248056fed3ca8d


If we look at our `docker` images again we no longer see newimagename:update 

In [128]:
docker images

REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE
nvidia/cuda         8.0-cudnn5-devel    31582c303549        5 weeks ago         1.776 GB
nvidia/cuda         latest              367795fb1051        5 weeks ago         1.615 GB
hello-world         latest              c54a2cc56cbb        5 months ago        1.848 kB


Finally, load the saved image into docker using the **`load`** command

In [129]:
docker load --input dockerimageexport.tar


Loaded image: newiamgename:update


In [131]:
docker images

REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE
newiamgename        update              5980494cc212        4 minutes ago       1.637 GB
nvidia/cuda         8.0-cudnn5-devel    31582c303549        5 weeks ago         1.776 GB
nvidia/cuda         latest              367795fb1051        5 weeks ago         1.615 GB
hello-world         latest              c54a2cc56cbb        5 months ago        1.848 kB


You should see the image `newimagename:update` listed!  

A few final words on saving vs exporting.  While the two methods are indeed similar in functionality, the difference is that *saving* an image will keep its history (i.e. all parent layers, tags, and versions) while *exporting* a container will squash its history producing a flattened single layer resource.

### Creating Images with Dockerfiles
Launching containers, making updates, and commiting the changes does work well however, it is quite manual.  To automate this image construction workflow `docker` provides a manifesto, called a [`dockerfile`](https://docs.docker.com/engine/reference/builder/), which is a text file that lists the build steps.  Dockerfiles are quite popular in the `docker` community and often docker files are exchanged rather than image tar-balls. While simple, there are a few common pitfalls when createing dockerfiles.  Read up on the [dockerfile best practices](https://docs.docker.com/engine/userguide/eng-image/dockerfile_best-practices/) for some excellent pointers that will save you lots of time.

Dockerfiles are quite simple lets create a dockerfile to build and updated version of the `nvidia/cuda:latest` image

In [158]:
cat << LINES > Dockerfile
FROM nvidia/cuda:latest
RUN apt-get update
ENTRYPOINT ["/usr/local/nvidia/bin/nvidia-smi"]
LINES


The [`FROM`](https://docs.docker.com/engine/reference/builder/#from) instruction sets the base image for subsequent instructions. As such, a valid `dockerfile` must have `FROM` as its first instruction. The image can be any valid image.  The [`RUN`](https://docs.docker.com/engine/reference/builder/#run) instruction will execute any commands in a new layer on top of the current image and commit the results. The resulting committed image will be used for the next step in the `dockerfile`.  Finally, the main purpose of a [`CMD`](https://docs.docker.com/engine/reference/builder/#cmd) is to provide defaults for an executing container. These defaults can include an executable, or they can omit the executable, in which case you must specify an [`ENTRYPOINT`](https://docs.docker.com/engine/reference/builder/#entrypoint) instruction as well.  For more information on how `CMD` and `ENTRYPOINT` interact see [here](https://docs.docker.com/engine/reference/builder/#understand-how-cmd-and-entrypoint-interact).

To build an image using this `dockerfile` we invoke the `docker` **`build`** command (this takes about 60 seconds)

In [162]:
docker build -t foo:bar .


Step 1 : FROM nvidia/cuda:latest
 ---> 367795fb1051
Step 2 : RUN apt-get update
 ---> Running in 4595c9266da4
Ign http://developer.download.nvidia.com  InRelease
Get:1 http://developer.download.nvidia.com  Release.gpg [819 B]
Get:2 http://developer.download.nvidia.com  Release [564 B]
Ign http://archive.ubuntu.com trusty InRelease
Get:3 http://archive.ubuntu.com trusty-updates InRelease [65.9 kB]
Get:4 http://developer.download.nvidia.com  Packages [107 kB]
Get:5 http://archive.ubuntu.com trusty-security InRelease [65.9 kB]
Get:6 http://archive.ubuntu.com trusty Release.gpg [933 B]
Get:7 http://archive.ubuntu.com trusty-updates/main Sources [479 kB]
Get:8 http://archive.ubuntu.com trusty-updates/restricted Sources [5921 B]
Get:9 http://archive.ubuntu.com trusty-updates/universe Sources [214 kB]
Get:10 http://archive.ubuntu.com trusty-updates/main amd64 Packages [1155 kB]
Get:11 http://archive.ubuntu.com trusty-updates/restricted amd64 Packages [20.4 kB]
Get:12 http://archive.ubuntu.co

In [163]:
docker images

REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE
foo                 bar                 33857ae05b40        26 seconds ago      1.637 GB
nvidia/cuda         8.0-cudnn5-devel    31582c303549        5 weeks ago         1.776 GB
nvidia/cuda         latest              367795fb1051        5 weeks ago         1.615 GB
hello-world         latest              c54a2cc56cbb        5 months ago        1.848 kB


You should now see the new image built with the dockerfile.  Don't hesitate to use the **`rmi`** command to clean up.

Dockerfiles are quite powerful and have many additional commands.  For reference here is the dockerfile for adding Jupyter Notebooks to an image 

In [ ]:
cat << LINES > JupyterDockerFile
FROM nvidia/caffe

RUN apt-get update && apt-get install -y \
        libzmq3-dev \
        python-dev \
        python-matplotlib \
        python-pandas \
        python-pip \
        python-sklearn && \
    rm -rf /var/lib/apt/lists/*

RUN pip install \
        ipykernel \
        jupyter && \
    python -m ipykernel.kernelspec

COPY jupyter_notebook_config.py /root/.jupyter/

COPY jupyter.sh /usr/local/bin

WORKDIR /data
VOLUME /data

EXPOSE 8888

CMD ["jupyter.sh"]

LINES

# New Section

In [3]:
nvidia-docker run --rm nvidia/cuda:8.0-cudnn5-devel nvidia-smi

Sat Nov 12 21:09:56 2016       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 367.57                 Driver Version: 367.57                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 0000:00:1E.0     Off |                    0 |
| N/A   45C    P8    28W / 149W |      0MiB / 11439MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [4]:
docker ps

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES


In [5]:
docker ps -a

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES


In [6]:
nvidia-docker run nvidia/cuda:8.0-cudnn5-devel nvidia-smi

Sat Nov 12 21:11:30 2016       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 367.57                 Driver Version: 367.57                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 0000:00:1E.0     Off |                    0 |
| N/A   45C    P8    28W / 149W |      0MiB / 11439MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [7]:
docker ps

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES


In [8]:
docker ps -a

CONTAINER ID        IMAGE                          COMMAND             CREATED             STATUS                      PORTS               NAMES
e310b48e17f9        nvidia/cuda:8.0-cudnn5-devel   "nvidia-smi"        12 seconds ago      Exited (0) 11 seconds ago                       backstabbing_engelbart


In [9]:
nvidia-docker start e310b48e17f9

e310b48e17f9


In [10]:
nvidia-docker start --help


Usage:	docker start [OPTIONS] CONTAINER [CONTAINER...]

Start one or more stopped containers

Options:
  -a, --attach               Attach STDOUT/STDERR and forward signals
      --detach-keys string   Override the key sequence for detaching a container
      --help                 Print usage
  -i, --interactive          Attach container's STDIN


In [11]:
nvidia-docker start -i e310b48e17f9

Sat Nov 12 21:16:36 2016       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 367.57                 Driver Version: 367.57                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 0000:00:1E.0     Off |                    0 |
| N/A   45C    P8    28W / 149W |      0MiB / 11439MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [12]:
nvidia-docker start -a e310b48e17f9

Sat Nov 12 21:17:01 2016       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 367.57                 Driver Version: 367.57                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 0000:00:1E.0     Off |                    0 |
| N/A   45C    P8    28W / 149W |      0MiB / 11439MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [13]:
nvidia-docker exec e310b48e17f9 echo "hello from nvidia"

Error response from daemon: Container e310b48e17f97c1103c53a3c91d008581019782cb523462c68dbbcebc4043e54 is not running


In [14]:
nvidia-docker run --interactive --tty -d e310b48e17f9 /bin/bash

Using default tag: latest
Pulling repository docker.io/library/e310b48e17f9
nvidia-docker | 2016/11/12 21:29:35 Error: image library/e310b48e17f9:latest not found


In [16]:
docker ps -a

CONTAINER ID        IMAGE                          COMMAND             CREATED             STATUS                      PORTS               NAMES
e310b48e17f9        nvidia/cuda:8.0-cudnn5-devel   "nvidia-smi"        20 minutes ago      Exited (0) 14 minutes ago                       backstabbing_engelbart


In [17]:
docker images

REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE
nvidia/cuda         8.0-cudnn5-devel    31582c303549        3 weeks ago         1.776 GB
nvidia/cuda         latest              367795fb1051        3 weeks ago         1.615 GB
hello-world         latest              c54a2cc56cbb        4 months ago        1.848 kB


In [18]:
nvidia-docker run --interactive --tty -d nvidia/cuda:8.0-cudnn5-devel /bin/bash

1868fbf74e0b9e613b4a41760f2b31257c280624a3b70d4705d4cb8c4f519150


In [19]:
docker ps 

CONTAINER ID        IMAGE                          COMMAND             CREATED             STATUS              PORTS               NAMES
1868fbf74e0b        nvidia/cuda:8.0-cudnn5-devel   "/bin/bash"         16 seconds ago      Up 15 seconds                           stupefied_bhabha


In [26]:
nvidia-docker exec 1868fbf74e0b ls /usr/local/cuda/extras

CUPTI
Debugger


# Appendix A
Here are some installation details for getting docker up and running from scratch ...

### Step 0 - GPU Driver
In order to get GPU access with in docker/nvidia-docker we need to make sure that the NVIDIA driver is available on the host system.  It's possible to obtain the appropriate device driver from either the standard [driver download](http://www.nvidia.com/download/index.aspx) page or via [CUDA installation](https://developer.nvidia.com/cuda-downloads).

### Step 1 - Docker Install
Once the NVIDIA device driver has been successfully installed, we need to install `docker` it self. The installation of docker is quite simple but it is just slightly different for each OS.  The steps for `docker` installation on Ubuntu 14.04 can be found [here](https://docs.docker.com/engine/installation/linux/ubuntulinux/).  Don't worry, the [`docker` docs](https://docs.docker.com/) have install instructions for many other operating systems including RedHat, CentOS, Debian, and so on.

Docker provides an official installation script via https://get.docker.com which can accessed via command-line using "`wget -qO-`" or "`curl -sSL`"

### Step 2 - NVIDIA Docker 
The final configuration step is to obtain the `nvidia-docker` plugin which properly exposes the GPU hardware and drivers for `docker` containers.  Official installation instructions for `nvidia-docker` for Ubuntu, CentOS, and other distributions can be found [here](https://github.com/NVIDIA/nvidia-docker)